In [1]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 36 mins
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,3 months and 3 days
H2O_cluster_name:,H2O_from_python_davidnunez_ai0opn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.292 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
model = h2o.load_model('/Users/davidnunez/Desktop/tesis/tesis-model/Training/train2021/unbalancedModelV4/StackedEnsemble_AllModels_1_AutoML_3_20230731_194426')

In [4]:
y = 'Protestas'
df_2022 = pd.read_csv('../df_dummizado_from_2021.csv', index_col=False)
df_2022 = df_2022[df_2022['date.year'] == 2022]

df_2022.to_csv('df_2022.csv', index = False, encoding='utf-8') # False: not include index
# df_2022 = df.s

test = h2o.H2OFrame.from_python(df_2022)
test[y] = test[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
drf = h2o.get_model('DRF_1_AutoML_6_20230731_225135')
# TOP: 'DRF_1_AutoML_8_20230801_04034'
# DRF_1_AutoML_7_20230731_234018

In [24]:
drf = h2o.get_model('DRF_1_AutoML_8_20230801_04034')
cm = drf.model_performance(test).confusion_matrix().to_list()

In [38]:
h2o.get_model('StackedEnsemble_BestOfFamily_1_AutoML_8_20230801_04034').model_performance(test).confusion_matrix()

,False,True,Error,Rate
False,16922.0,1744.0,0.0934,(1744.0/18666.0)
True,1479.0,1839.0,0.4458,(1479.0/3318.0)
Total,18401.0,3583.0,0.1466,(3223.0/21984.0)


In [30]:
cm[1][1] / (cm[1][0] + cm[1][1])


0.5533453887884268

In [37]:
type(drf)

h2o.estimators.random_forest.H2ORandomForestEstimator

In [36]:
drf.model_performance(test).recall()[0][0]


0.012416331904275081